1. Initialization
    * Initialize variables
    * Load Scripts
2. Applying machine learning model to predict sgRNA activity
    * Find all sgRNAs in genomic regions of interest 
    * Predicting sgRNA activity
3. Construct sgRNA libraries
    * Score sgRNAs for off-target potential
* Pick the top sgRNAs for a library, given predicted activity scores and off-target filtering

# 1. Initialization
## Initialize variables and load scripts

In [ ]:
#Modif this variable to add genes for which you want to find guideRNAs
GENES_LIST_TO_FIND_GUIDES = ['APOL1','APOL3','PRRC2A']

#modify below variable to point to the base directory containing large genome data files
#LARGE_FILE_DIR= '../../../../data/'
LARGE_FILE_DIR= './'


FASTA_FILE_OF_GENOME= LARGE_FILE_DIR + 'large_data_files/hg19.fa'
GTF_FILE_FROM_GENCODE = LARGE_FILE_DIR + 'large_data_files/gencode.v19.annotation.gtf'

TSS_TABLE_PATH='data_files/human_tssTable.txt'
P1P2_TABLE_PATH='data_files/human_p1p2Table.txt'

FANTOM_TSS_ANNOTATION_BED= LARGE_FILE_DIR + 'large_data_files/TSS_human.sorted.bed.gz'
HGNC_SYMBOL_LOOKUP_TABLE= LARGE_FILE_DIR + 'large_data_files/hgnc_complete_set_2020-08-01.txt'

#spreadsheet containing the lab experiment data to train the model
IGEM_EXCEL_FILE= 'data_files/Sam_Perli_qRT_pcr_data_per_gene.xlsx'


PICKLE_FILE = 'igem_v1_estimator'
TRANSFORMED_PARAM_HEADER='igem_v1_transformed_param_header'

PREDICTED_SCORE_TABLE='igem_v1_predicted_score_table'
TEMP_FASTQ_FILE='igem_v1_temp_fastq_file'


%run sgRNA_learning.py

# 2. Applying machine learning model to predict sgRNA activity

In [ ]:
#Here we load our trained model
import _pickle as cPickle

#load tssTable, p1p2Table, genome sequence, chromatin data
tssTable = pd.read_csv(TSS_TABLE_PATH,sep='\t', index_col=[0,1])

p1p2Table = pd.read_csv(P1P2_TABLE_PATH,sep='\t', header=0, index_col=[0,1])
p1p2Table['primary TSS'] = p1p2Table['primary TSS'].apply(lambda tupString: (int(float(tupString.strip('()').split(', ')[0])), int(float(tupString.strip('()').split(', ')[1]))))
p1p2Table['secondary TSS'] = p1p2Table['secondary TSS'].apply(lambda tupString: (int(float(tupString.strip('()').split(', ')[0])),int(float(tupString.strip('()').split(', ')[1]))))

genomeDict = loadGenomeAsDict(FASTA_FILE_OF_GENOME)

bwhandleDict = {'dnase':BigWigFile(open(LARGE_FILE_DIR + 'large_data_files/wgEncodeOpenChromDnaseK562BaseOverlapSignalV2.bigWig','rb')),
'faire':BigWigFile(open(LARGE_FILE_DIR + 'large_data_files/wgEncodeOpenChromFaireK562Sig.bigWig','rb')),
'mnase':BigWigFile(open(LARGE_FILE_DIR + 'large_data_files/wgEncodeSydhNsomeK562Sig.bigWig','rb'))}

#load sgRNA prediction model saved after the parameter fitting step
with open(PICKLE_FILE,'rb') as infile:
    fitTable, estimators, scaler, reg, (geneFold_train, geneFold_test) = cPickle.load(infile)
    
#transformedParamHeader = pd.read_csv(TRANSFORMED_PARAM_HEADER,sep='\t')

#iGEM read the binary file
#transformedParamHeader = pd.read_csv(TRANSFORMED_PARAM_HEADER,sep='\t')

with open(TRANSFORMED_PARAM_HEADER,'rb') as paraminfile:
    transformedParamHeader = cPickle.load(paraminfile)

transformedParams_train = transformedParamHeader

## Find all sgRNAs in genomic regions of interest 

In [ ]:
#Create gene and sgRna table
libraryTable_new, sgInfoTable_new = findAllGuides(p1p2Table, genomeDict, (-25,500))

#Filter the genes  that we are interested in
libraryTable_new = libraryTable_new[ (libraryTable_new['gene'].isin(GENES_LIST_TO_FIND_GUIDES))]
sgInfoTable_new = parseAllSgIds(libraryTable_new)


In [ ]:
libraryTable_new.head()


## Predicting sgRNA activity

In [ ]:
#calculate parameters for new sgRNAs
paramTable_new = generateTypicalParamTableEx(libraryTable_new, sgInfoTable_new, tssTable, p1p2Table, genomeDict, bwhandleDict)

In [ ]:
#transform and predict scores according to sgRNA prediction model
transformedParams_new = transformParams(paramTable_new, fitTable, estimators)

#reconcile any differences in column headers generated by automated binning
colTups = []
for (l1, l2), col in transformedParams_new.iteritems():
    colTups.append((l1,str(l2)))
transformedParams_new.columns = pd.MultiIndex.from_tuples(colTups)
#iGEM in python 3 .loc with missing column headers is giving issues. So changing it to reindex # can use transformedParams_train if running sequentially otherwise use transformedParamHeader after running above step
predictedScores_new = pd.Series(reg.predict(scaler.transform(transformedParams_new.reindex(columns=transformedParams_train.columns).fillna(0).values)), index=transformedParams_new.index)

In [ ]:
predictedScores_new.head()

In [ ]:
predictedScores_new.to_csv(PREDICTED_SCORE_TABLE, sep='\t')

# 3. Construct sgRNA libraries
## Score sgRNAs for off-target potential

In [ ]:
#There are many ways to score sgRNAs as off-target; below is one listed one method that is simple and flexible,
#but ignores gapped alignments, alternate PAMs, and uses bowtie which may not be maximally sensitive in all cases

In [ ]:
#iGEM the sequence length can be greather 22. So just added more pluses and adjusteed its length equal to sequence length.
#iGEM Revisit to fix phred length properly.

#output all sequences to a temporary FASTQ file for running bowtie alignment
def outputTempBowtieFastq(libraryTable, outputFileName):
    phredString = 'I4!=======44444++++++++++++++++' #weighting for how impactful mismatches are along sgRNA sequence 
    with open(outputFileName,'w') as outfile:
        for name, row in libraryTable.iterrows():
            outfile.write('@' + name + '\n')
            outfile.write('CCN' + str(Seq.Seq(row['sequence'][1:]).reverse_complement()) + '\n')
            outfile.write('+\n')
            outfile.write(phredString[0:3+len(str(Seq.Seq(row['sequence'][1:]).reverse_complement()))] + '\n')
            
outputTempBowtieFastq(libraryTable_new, TEMP_FASTQ_FILE)

In [ ]:
import subprocess
fqFile = TEMP_FASTQ_FILE

#specifying a list of parameters to run bowtie with
#each tuple contains
# *the mismatch threshold below which a site is considered a potential off-target (higher is more stringent)
# *the number of sites allowed (1 is minimum since each sgRNA should have one true site in genome)
# *the genome index against which to align the sgRNA sequences; these can be custom built to only consider sites near TSSs
# *a name for the bowtie run to create appropriately named output files

#iGEM
#alignmentList = [(39,1,'~/indices/hg19.ensemblTSSflank500b','39_nearTSS'),
#                (31,1,'~/indices/hg19.ensemblTSSflank500b','31_nearTSS'),
#                (21,1,'~/indices/hg19.maskChrMandPAR','21_genome'),
#                (31,2,'~/indices/hg19.ensemblTSSflank500b','31_2_nearTSS'),
#                (31,3,'~/indices/hg19.ensemblTSSflank500b','31_3_nearTSS')]

#iGEM   ChrM and PAR are vary small part of hg19. So running alignment for entire genome.

alignmentList = [(39,1,LARGE_FILE_DIR+'large_data_files/indices/hg19.ensemblTSSflank500b','39_nearTSS'),
                (31,1,LARGE_FILE_DIR+'large_data_files/indices/hg19.ensemblTSSflank500b','31_nearTSS'),
                #(21,1,'/data/large_data_files/indices/hg19_maskChrMandPAR','21_genome'),
                (21,1,LARGE_FILE_DIR+'large_data_files/indices/hg19','21_genome'),
                (31,2,LARGE_FILE_DIR+'large_data_files/indices/hg19.ensemblTSSflank500b','31_2_nearTSS'),
                (31,3,LARGE_FILE_DIR+'large_data_files/indices/hg19.ensemblTSSflank500b','31_3_nearTSS')]
import os
import errno
if not os.path.exists('bowtie_output'):
    os.makedirs('bowtie_output')

alignmentColumns = []
for btThreshold, mflag, bowtieIndex, runname in alignmentList:

    alignedFile = 'bowtie_output/' + runname + '_aligned.txt'
    unalignedFile = 'bowtie_output/' + runname + '_unaligned.fq'
    maxFile = 'bowtie_output/' + runname + '_max.fq'
    
    bowtieString = 'bowtie -n 3 -l 15 -e '+str(btThreshold)+' -m ' + str(mflag) + ' --nomaqround -a --tryhard -p 16 --chunkmbs 256 ' + bowtieIndex + ' --suppress 5,6,7 --un ' + unalignedFile + ' --max ' + maxFile + ' '+ ' -q '+fqFile+' '+ alignedFile
    print (bowtieString)
    print (subprocess.call(bowtieString, shell=True))

    #parse through the file of sgRNAs that exceeded "m", the maximum allowable alignments, and mark "True" any that are found
    sgsAligning = set()
    
    try:
        with open(maxFile) as infile:
            sgsAligning = set()
            for i, line in enumerate(infile):
                if i%4 == 0: #id line
                    sgsAligning.add(line.strip()[1:])
    except OSError as e:
        if e.errno != errno.ENOENT:
            raise

    alignmentColumns.append(libraryTable_new.apply(lambda row: row.name in sgsAligning, axis=1))
#iGEM zip is an object in python3    
#collate results into a table, and flip the boolean values to yield the sgRNAs that passed filter as True
alignmentTable = pd.concat(alignmentColumns,axis=1, keys=list(zip(*alignmentList))[3]).ne(True)

## Pick the top sgRNAs for a library, given predicted activity scores and off-target filtering

In [ ]:
#combine all generated data into one master table
predictedScores_new.name = 'predicted score'
v2Table = pd.concat((libraryTable_new, predictedScores_new, alignmentTable, sgInfoTable_new), axis=1, keys=['library table v2', 'predicted score', 'off-target filters', 'sgRNA info'])

In [ ]:
v2Table

In [ ]:
import re
#for our pCRISPRi/a-v2 vector, we append flanking sequences to each sgRNA sequence for cloning and require the oligo to contain
#exactly 1 BstXI and BlpI site each for cloning, and exactly 0 SbfI sites for sequencing sample preparation
restrictionSites = {re.compile('CCA......TGG'):1,
                   re.compile('GCT.AGC'):1,
                   re.compile('CCTGCAGG'):0}

def matchREsites(sequence, REdict):
    seq = sequence.upper()
#iGEM in python 3 dict.iteritems is not present. So replace with dict.items        
#    for resite, numMatchesExpected in restrictionSites.iteritems():
    for resite, numMatchesExpected in restrictionSites.items():
        if len(resite.findall(seq)) != numMatchesExpected:
            return False
        
    return True

def checkOverlaps(leftPosition, acceptedLeftPositions, nonoverlapMin):
    for pos in acceptedLeftPositions:
        if abs(pos - leftPosition) < nonoverlapMin:
            return False
    return True

In [ ]:
#flanking sequences
upstreamConstant = 'CCACCTTGTTG'
downstreamConstant = 'GTTTAAGAGCTAAGCTG'

#minimum overlap between two sgRNAs targeting the same TSS
nonoverlapMin = 3

#number of sgRNAs to pick per gene/TSS
sgRNAsToPick = 10

#iGEM TODO need to enable 21_genome as well. temporarily disabled

#list of off-target filter (or combinations of filters) levels, matching the names in the alignment table above
offTargetLevels = [['31_nearTSS', '21_genome'],
                  ['31_nearTSS'],
                  ['21_genome'],
                  ['31_2_nearTSS'],
                  ['31_3_nearTSS']]

#offTargetLevels = [ ['31_nearTSS'],
#                  ['31_2_nearTSS'],
#                  ['31_3_nearTSS']]


#for each gene/TSS, go through each sgRNA in descending order of predicted score
#if an sgRNA passes the restriction site, overlap, and off-target filters, accept it into the library
#if the number of sgRNAs accepted is less than sgRNAsToPick, reduce off-target stringency by one and continue
v2Groups = v2Table.groupby([('library table v2','gene'),('library table v2','transcripts')])
newSgIds = []
unfinishedTss = []
for (gene, transcript), group in v2Groups:
    geneSgIds = []
    geneLeftPositions = []
    empiricalSgIds = dict()
    
    stringency = 0
#iGEM use sort_values instead of sort    
    while len(geneSgIds) < sgRNAsToPick and stringency < len(offTargetLevels):
        for sgId_v2, row in group.sort_values(('predicted score','predicted score'), ascending=False).iterrows():
            oligoSeq = upstreamConstant + row[('library table v2','sequence')] + downstreamConstant
            leftPos = row[('sgRNA info', 'position')] - (23 if row[('sgRNA info', 'strand')] == '-' else 0)
            if len(geneSgIds) < sgRNAsToPick and row['off-target filters'].loc[offTargetLevels[stringency]].all() \
                and matchREsites(oligoSeq, restrictionSites) \
                and checkOverlaps(leftPos, geneLeftPositions, nonoverlapMin):
                geneSgIds.append((sgId_v2,
                                  gene,transcript,
                                  row[('library table v2','sequence')], oligoSeq,
                                  row[('predicted score','predicted score')], np.nan,
                                 stringency))
                geneLeftPositions.append(leftPos)
                
        stringency += 1
            
    if len(geneSgIds) < sgRNAsToPick:
        unfinishedTss.append((gene, transcript)) #if the number of accepted sgRNAs is still less than sgRNAsToPick, discard gene
    else:
        newSgIds.extend(geneSgIds)
        
libraryTable_complete = pd.DataFrame(newSgIds, columns = ['sgID', 'gene', 'transcript','protospacer sequence', 'oligo sequence',
 'predicted score', 'empirical score', 'off-target stringency']).set_index('sgID')

In [ ]:
unfinishedTss

In [ ]:
#number of sgRNAs accepted at each stringency level
#iGEM newLibraryTable is not defined
#newLibraryTable.groupby('off-target stringency').agg(len).iloc[:,0]
libraryTable_complete.groupby('off-target stringency').agg(len).iloc[:,0]

In [ ]:
#number of TSSs with fewer than required number of sgRNAs (and thus not included in the library)
print (len(unfinishedTss))

In [ ]:
libraryTable_complete